<a href="https://colab.research.google.com/github/CChancyy/MyPrivateTesting/blob/main/Copy_1_of_Orbital_2023_Telegram_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install python-telegram-bot
!pip install nest_asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# Workaround for nested event loops in Jupyter
# As Jupyter notebooks already have a running loop via Tornado and the asyncio lib does not allow a nested loop, we need to use a separate package called nest_asyncio
import nest_asyncio
import asyncio

nest_asyncio.apply() 

In [5]:
# Enable logging
import logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)

In [6]:
# Your token goes here
token = "5822080229:AAGQV8VM3SqQ5WfB7P9CNG9sf_q4PodHdy4"

# Our First Bot!

We will learn the basics here.

In [7]:
from telegram import Update
from telegram.constants import ParseMode
from telegram import InlineQueryResultArticle, InputTextMessageContent
from telegram.ext import filters, MessageHandler, ApplicationBuilder, CommandHandler, ContextTypes, InlineQueryHandler
import math
from random import random
import datetime

In [ ]:
# Define a few command handlers. These usually take the two arguments update and context.
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    await context.bot.send_message(chat_id=update.effective_chat.id, text=f'Hi {user.mention_markdown_v2()}\!\nYou can talk to me', parse_mode=ParseMode.MARKDOWN_V2)

async def echo(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Echoes the user's message."""
    # TODO 1 Get your feet wet! Try this first
    # Hint: use `text=update.message.text`
    # await context.bot.send_message(chat_id=update.effective_chat.id, text = update.message.text)
    # TODO 2 An alternative approach
    # await update.message.reply_text(update.message.text)

async def help(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Send a message when the command /help is issued."""
    await context.bot.send_message(chat_id=update.effective_chat.id, text="You are saved!")
    #TODO 3 Register my handlers


async def unknown(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """In the case of an unknown command"""
    #TODO 4
    await context.bot.send_message(chat_id=update.effective_chat.id, text="Sorry, I do not understand what you are talking about.") 

async def quadratic(update, context) -> None:
    # This will give us all the words in the message, which will be something like "/quadratic 1 2 3"
    # Hint: You can use the `split()` method of a Python string
    # TODO 5
    split_string = update.message.text.split(" ")
    ## now it becomes [quaratic, 'a', 'b', 'c']
    coefficient = split_string[1:]
    ## now it become ['a', 'b', 'c'] need to convert string into int 
    a = int(coefficient[0])
    b = int(coefficient[1])
    c = int(coefficient[2])

    disc = math.sqrt(b ** 2 - 4 * a * c)

    sol1 = (-b + disc) / ( 2 * a )
    sol2 = (-b - disc) / ( 2 * a )
    await context.bot.send_message(chat_id=update.effective_chat.id, text=f"Your roots are {sol1} and {sol2}")
    
async def cat(update, context) -> None: 
    timestamp = datetime.datetime.now().isoformat()
    url = f"https://cataas.com/cat?a={timestamp}" # As Telegram caches the URL
    #Hint: You can use the `reply_photo()` method
    # TODO 6
    pass

# Bonus content - inline mode. To test this out, type `/setinline` in @BotFather and type @<your bot's name> in any of your other chats to talk to your bot
async def inline_caps(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Capitalises the message."""
    query = update.inline_query.query
    if not query:
        return
    results = []
    results.append(
        InlineQueryResultArticle(
            id=query.upper(),
            title='Caps',
            input_message_content=InputTextMessageContent(query.upper())
        )
    )
    await context.bot.answer_inline_query(update.inline_query.id, results)

async def error(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Log errors caused by updates."""
    logger.error('Update "%s" caused error "%s"', update, context.error)

def main() -> None:
    """Start the bot."""
    application = ApplicationBuilder().token(token).build()
    
    # Create your handlers here
    start_handler = CommandHandler('start', start)
    echo_handler = MessageHandler(filters.TEXT & (~filters.COMMAND), echo)
    help_handler = CommandHandler ('help', help)
    quadratic_handler = CommandHandler ('quadratic', quadratic)
    unknown_handler = MessageHandler(filters.COMMAND, unknown)
    

    # Register your handlers here
    application.add_handler(start_handler)
    application.add_handler(echo_handler)
    application.add_handler(help_handler)
    application.add_handler(quadratic_handler)
    
    application.add_handler(unknown_handler)
    
    # Error handler
    application.add_error_handler(error)
    
    # Add your quadratic and cat handlers here
    # TODO 7

    # We pass in the Telegram event loop as a nested event loop here - only for Jupyter
    # Python version < 3.7
    # loop = asyncio.get_event_loop()
    # loop.create_task(application.run_polling())
    # Python version >= 3.7
    asyncio.run(application.run_polling())

main()

## answer is here: https://github.com/parth-io/orbital-2023-telegram-bots/tree/master/req-resp-bot

# An Enhanced Bot

Let's learn how to set up polling. First, let's check that the website we want to listen for updates on is up and running.

In [ ]:
import requests

url = "http://206.189.92.77/"

resp = requests.get(url)
resp.text

'<!DOCTYPE html>\n<html lang="en">\n    <head>\n        <meta charset="UTF-8" />\n        <meta http-equiv="X-UA-Compatible" content="IE=edge" />\n        <meta name="viewport" content="width=device-width, initial-scale=1.0" />\n        <title>Server side alerts</title>\n    </head>\n    <body>\n        <h1>Colour of the day: Blue</h1>\n    </body>\n</html>\n'

Try implementing a basic polling mechanism here.


In [ ]:
import time
import requests
from telegram import Bot, Update
from telegram.ext import ApplicationBuilder, ContextTypes

async def manual_poll(application):
    while True:
        r = requests.get("http://206.189.92.77")
        # TODO 1 add your implementation here
        time.sleep(1) # sleep for a while, to avoid overloading the server

async def error(update: Update, context: ContextTypes.DEFAULT_TYPE) -> None:
    """Log errors caused by updates."""
    logger.error('Update "%s" caused error "%s"', update, context.error)

def main() -> None:
    """Start the bot."""
    application = ApplicationBuilder().token(token).build()
    
    # Error handler
    application.add_error_handler(error)
    
    # We pass in the Telegram event loop as a nested event loop here - only for Jupyter
    # Python version < 3.7
    # loop = asyncio.get_event_loop()
    # loop.create_task()
    # Python version >= 3.7
    asyncio.run(manual_poll(application)) # TODO 2 what goes here :)
    # Note that as the library's polling loop is not being used,
    # asyncio.run() will have to be used even outside of Jupyter, unlike the previous bot

main()


KeyboardInterrupt: ignored

# ChatGPT Time

The obligatory ChatGPT session